In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import psycopg2
from datetime import datetime

In [2]:
response = requests.get('https://ru.investing.com/crypto')
print(response.status_code) # We are checking the connection to the site. If 200, then everything is OK.

200


In [3]:
soup = BeautifulSoup(response.text, 'lxml') # uploading the HTML code

In [4]:
tbody = soup.find("tbody", class_="datatable-v2_body__8TXQk").find_all("tr") # select the table that we need

In [5]:
list_temp = [] # an empty list for data
for row in tbody:
    temp = row.get_text("|", strip=True).split('|') # выводим чистый текст
    temp = temp[:3] + temp[6:] 
    list_temp.append(temp)

In [6]:
list_btc = list_temp[0] # since a table with data from different currencies is displayed, we only output BTC. 

In [7]:
list_btc # let's see what we've deduced

['Биткоин', 'BTC', '97.360,6', '-1,32%', '-6,43%', '$1,93T']

In [8]:
conn = psycopg2.connect(
    dbname='cripto',
    user='postgres',
    password='postgres',  
    host='postgres', 
    port=5432
)
# setting up a connection to the database

In [9]:
cur = conn.cursor() # Opening the session

In [10]:
current_date = datetime.now() #We add the request completion date and process the rest of the data
current_name = list_btc[0]
current_tag = list_btc[1]
current_price = float(list_btc[2].replace('.', '').replace(',', '.'))
current_dif_1d = float(list_btc[3].replace('.', '').replace(',', '.').replace('%', ''))
current_dif_7d = float(list_btc[4].replace('.', '').replace(',', '.').replace('%', ''))
current_capital = int(float(list_btc[5].replace('$', '').replace(',', '.').replace('T', '')) * 1e12)

In [11]:
try:
    insert_sql = '''
    INSERT INTO btc_price (date, name, tag, price, dif_1d, dif_7d, capital) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    '''
    cur.execute(insert_sql, (current_date, current_name, current_tag, current_price, current_dif_1d, current_dif_7d, current_capital))
    conn.commit()
    print('Запись успешно добавлена!')
except Exception as e:
    print('Ошибка:', e) # uploading data to the database


Запись успешно добавлена!


In [12]:
cur.close() # Closing the session

In [13]:
#conn.rollback() # just in case =)

In [14]:
list_temp # This is what the table of all coins we are uploading looks like.

[['Биткоин', 'BTC', '97.360,6', '-1,32%', '-6,43%', '$1,93T'],
 ['Эфириум', 'ETH', '2.742,40', '-3,06%', '-15,71%', '$331,51B'],
 ['Tether USDt', 'USDT', '1,0003', '-0,03%', '+0,04%', '$141,42B'],
 ['XRP', 'XRP', '2,3816', '-2,86%', '-22,47%', '$137,68B'],
 ['Solana', 'SOL', '197,283', '-2,46%', '-16,09%', '$96,33B'],
 ['BNB', 'BNB', '584,80', '+0,96%', '-13,63%', '$83,52B'],
 ['USDC', 'USDC', '0,9996', '-0,01%', '+0,02%', '$55,91B'],
 ['Dogecoin', 'DOGE', '0,253357', '-3,89%', '-22,41%', '$37,55B'],
 ['Cardano', 'ADA', '0,7297', '-4,17%', '-23,30%', '$25,73B'],
 ['TRON', 'TRX', '0,227689', '-0,19%', '-10,44%', '$19,63B'],
 ['Chainlink', 'LINK', '19,24', '-2,88%', '-22,95%', '$12,30B'],
 ['Avalanche', 'AVAX', '25,30', '-5,22%', '-26,44%', '$10,43B'],
 ['Stellar', 'XLM', '0,32893', '-2,99%', '-21,32%', '$10,08B'],
 ['Sui', 'SUI', '3,2311', '-6,58%', '-21,99%', '$10,02B'],
 ['Toncoin', 'TON', '3,7774', '-3,66%', '-23,02%', '$9,37B'],
 ['Hedera', 'HBAR', '0,23658', '-4,55%', '-23,95%', '$

In [15]:
list([current_date, 
      current_name, 
      current_tag, 
      current_price, 
      current_dif_1d, 
      current_dif_7d, 
      current_capital]) # This is what we uploaded to the database.

[datetime.datetime(2025, 2, 7, 11, 23, 2, 97333),
 'Биткоин',
 'BTC',
 97360.6,
 -1.32,
 -6.43,
 1930000000000]